In [ ]:
import os
import pandas as pd

data = pd.read_csv('DNNProject/dataset/Data_Entry_2017.csv')
data = data.rename(columns={'Finding Label': 'Finding Labels'})

def filterData(data):
    filtered_data = data[
    ~data['Finding Labels'].str.contains('Hernia', na=False) &
    ~data['Finding Labels'].str.contains('Pneumonia', na=False) &
    ~data['Finding Labels'].str.contains('Edema', na=False) &
    ~data['Finding Labels'].str.contains('Cardiomegaly', na=False) &
    ~data['Finding Labels'].str.contains('Consolidation', na=False) &
    ~data['Finding Labels'].str.contains('Pleural_Thickening', na=False) &
    ~data['Finding Labels'].str.contains('Fibrosis', na=False) &
    ~data['Finding Labels'].str.contains('Emphysema', na=False) &
    #~data['Finding Labels'].str.contains('Nodule', na=False) &
    ~data['Finding Labels'].str.contains('Pneumothorax', na=False) &
    ~data['Finding Labels'].str.contains('Mass', na=False) &
    ~data['Finding Labels'].str.contains('No Finding', na=False) &
    ~data['Finding Labels'].str.contains(r'\|', na=False)
    ]
    return filtered_data

filtered_data = filterData(data)
filtered_data = filtered_data[['Image Index', 'Finding Labels']]


In [ ]:
def find_image_path_nested(image_index, base_dir):
    #global subfolders_with_valid_paths, subfolders_without_valid_paths
    subfolders = sorted(os.listdir(base_dir))  # Sort subfolders for consistency
    found_path = None  # Initialize as None

    for subfolder in subfolders:
        subfolder_path = os.path.join(base_dir, subfolder)
        if os.path.isdir(subfolder_path):  # Check if it's a directory
            # Look for a nested 'images' folder
            nested_images_path = os.path.join(subfolder_path, 'images')
            if os.path.exists(nested_images_path) and os.path.isdir(nested_images_path):
                # Check if the image exists in the nested folder
                image_path = os.path.join(nested_images_path, image_index)
                if os.path.exists(image_path):
                    #subfolders_with_valid_paths.add(subfolder)  # Mark subfolder as valid
                    found_path = image_path
                    break  # Stop searching once the image is found
            #else:
                #subfolders_without_valid_paths.add(subfolder)  # Mark as invalid
    return found_path


In [ ]:
base_dir = 'DNNProject/dataset'

subfolders_with_valid_paths = set()
subfolders_without_valid_paths = set()

filtered_data['Image Path'] = filtered_data['Image Index'].apply(lambda x: find_image_path_nested(x, base_dir))


print(f"Number of None paths: {filtered_data['Image Path'].isna().sum()}")
print(f"Number of valid paths: {filtered_data['Image Path'].notna().sum()}")


filtered_data = filtered_data.dropna(subset=['Image Path'])


print(filtered_data.head())

In [ ]:

# %%
import matplotlib.pyplot as plt
from PIL import Image


class_counts = filtered_data['Finding Labels'].value_counts()

# Plot the counts
plt.figure(figsize=(10, 6))
class_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Samples per Class', fontsize=16)
plt.xlabel('Class', fontsize=14)
plt.ylabel('Number of Samples', fontsize=14)
plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
#plt.savefig('results/class_dist.png', dpi=300)
plt.show()


In [ ]:
# %%
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

train_df, valid_df = train_test_split(
    filtered_data,
    test_size=0.20, 
    random_state=42,
    stratify=filtered_data['Finding Labels'] 
)

print("Training set distribution:")
print(train_df['Finding Labels'].value_counts())
print("\nValidation set distribution:")
print(valid_df['Finding Labels'].value_counts())


label_to_index = {label: idx for idx, label in enumerate(filtered_data['Finding Labels'].unique())}
index_to_label = {idx: label for label, idx in label_to_index.items()}  # Reverse mapping

train_df['Finding Labels'] = train_df['Finding Labels'].map(label_to_index)
valid_df['Finding Labels'] = valid_df['Finding Labels'].map(label_to_index)

print("\nLabel to Index Mapping:")
for label, idx in label_to_index.items():
    print(f"Label: {label}, Index: {idx}")


In [ ]:
# %%
def resample_data(train_df, label_name,label_to_index, 
                  downsample=True, upsample=True, 
                  target_downsample_count=500, target_upsample_count=160):
   
    
    no_finding_label = label_to_index[label_name]
    no_finding_samples = train_df[train_df['Finding Labels'] == no_finding_label]
    other_classes = train_df[train_df['Finding Labels'] != no_finding_label]

  
    if downsample:
        print(f"Downsampling {label_name} to {target_downsample_count} samples.")
        downsampled_no_finding = no_finding_samples.sample(
            n=target_downsample_count, random_state=42
        )
    else:
        downsampled_no_finding = no_finding_samples

  
    upsampled_dfs = []
    if upsample:
        class_counts = other_classes['Finding Labels'].value_counts()
        classes_to_upsample = class_counts[class_counts < target_upsample_count].index
        print(f"Upsampling classes: {classes_to_upsample} to {target_upsample_count} samples each.")

        for class_label in classes_to_upsample:
            class_rows = other_classes[other_classes['Finding Labels'] == class_label]
            upsampled_rows = class_rows.sample(
                n=target_upsample_count, replace=True, random_state=42
            )
            upsampled_dfs.append(upsampled_rows)

   
    remaining_classes = other_classes[~other_classes['Finding Labels'].isin(classes_to_upsample)] if upsample else other_classes
    final_train_df = pd.concat([downsampled_no_finding, remaining_classes] + upsampled_dfs)

   
    final_train_df = final_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

    print("Resampling completed.")
    return final_train_df

"""train_df = resample_data(
    train_df=train_df,
    label_name = 'Infiltration',
    label_to_index=label_to_index,
    downsample=True,
    upsample=False,  # Disable upsampling
    target_downsample_count=300
)"""

train_df = resample_data(
    train_df=train_df,
    label_name = 'Infiltration',
    label_to_index=label_to_index,
    downsample=True,
    upsample=False,  # Disable upsampling
    target_downsample_count=4000
)

class_counts_after_sampling = train_df['Finding Labels'].value_counts()

print("Label | Index | Number of Samples")
print("-" * 40)
for label, idx in label_to_index.items():
    count = class_counts_after_sampling[idx] if idx in class_counts_after_sampling else 0
    print(f"{label:<15} | {idx:<5} | {count:<10}")


In [ ]:
from PIL import Image
import torch
from torch.utils.data import Dataset
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, transform=None):
    
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):

        img_path = self.dataframe.iloc[idx]['Image Path']
        label = self.dataframe.iloc[idx]['Finding Labels']

    
        image = Image.open(img_path).convert("RGB")  

        if self.transform:
            image = self.transform(image)

        label_tensor = torch.tensor(label, dtype=torch.long)

        return image, label_tensor


In [ ]:
# %%
from torch.utils.data import DataLoader
from torchvision import transforms

#
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print(train_transforms.transforms)


train_dataset = ChestXrayDataset(train_df, transform=train_transforms)
val_dataset = ChestXrayDataset(valid_df, transform=val_transforms)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# %%
import torch.nn as nn
import torch.nn.functional as F
import torch

class ResBlocks(nn.Module):
    def __init__(self, in_channels, out_channels, use_bottleneck=False, downsample=False):
        super(ResBlocks, self).__init__()
        stride = 2 if downsample else 1

        if use_bottleneck:
            self.block = self.bottleneck_block(in_channels, out_channels, stride)
        else:
            self.block = self.residual_block(in_channels, out_channels, stride)

        # Oppdater shortcut for å bruke riktig stride hvis det er downsample
        if downsample or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

    def residual_block(self, in_channels, out_channels, stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def bottleneck_block(self, in_channels, out_channels, stride):
        bottleneck_channels = out_channels // 4
        return nn.Sequential(
            nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(bottleneck_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(bottleneck_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.block(x) + self.shortcut(x))


# %%
class ResNet(nn.Module):
    def __init__(self, num_classes, resnetModel='ResNet50'):
        super(ResNet, self).__init__()
        self.resnetModel = resnetModel
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.num_classes = num_classes

        if resnetModel == 'ResNet34':
            self.res34layers()
        elif resnetModel == 'ResNet50':
            self.res50layers()
        else:
            print('Model not implemented')

        
    def res34layers(self):
        self.conv2_x = self._make_layer(64, 64, n_blocks=3, use_bottleneck=False)
        self.conv3_x = self._make_layer(64, 128, n_blocks=4, use_bottleneck=False, downsample=True)
        self.conv4_x = self._make_layer(128, 256, n_blocks=6, use_bottleneck=False, downsample=True)
        self.conv5_x = self._make_layer(256, 512, n_blocks=3, use_bottleneck=False, downsample=True)

        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(512, self.num_classes)
    def res50layers(self):
        self.conv2_x = self._make_layer(64, 256, n_blocks=3, use_bottleneck=True)
        self.conv3_x = self._make_layer(256, 512, n_blocks=4, use_bottleneck=True, downsample=True)
        self.conv4_x = self._make_layer(512, 1024, n_blocks=6, use_bottleneck=True, downsample=True)
        self.conv5_x = self._make_layer(1024, 2048, n_blocks=3, use_bottleneck=True, downsample=True)
        self.global_avg_pooling = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(2048, self.num_classes)

    def _make_layer(self, in_channels, out_channels, n_blocks, use_bottleneck, downsample=False):
        layers = [ResBlocks(in_channels, out_channels, use_bottleneck, downsample)]
        for _ in range(1, n_blocks):
            layers.append(ResBlocks(out_channels, out_channels, use_bottleneck))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)

        x = self.global_avg_pooling(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)

        x = self.fc(x)
        return x



In [ ]:
# %%

from sklearn.utils.class_weight import compute_class_weight
num_classes = len(train_df['Finding Labels'].unique())
model = ResNet(num_classes=num_classes, resnetModel='ResNet34')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_weights = compute_class_weight('balanced', classes=train_df['Finding Labels'].unique(), y=train_df['Finding Labels'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights,label_smoothing=0.01)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.001, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3, mode='min')
model = model.to(device)


# %%
from tqdm.notebook import tqdm
# Training Loop
num_epochs = 50
avg_train_loss = []
avg_train_acc = []
avg_val_loss = []
avg_val_acc = []

with tqdm(total=num_epochs, desc="Training Progress", unit="epoch") as epoch_pbar:
    print("Init training")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        total_train = 0
        correct_train = 0

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_train_loss = running_loss / len(train_loader)
        avg_train_loss.append(epoch_train_loss)

        epoch_train_acc = correct_train / total_train
        avg_train_acc.append(epoch_train_acc)

        #print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train acc: {epoch_train_acc:.4f}")

        # Validation Phase
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0

        all_preds = []
        all_probs = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                probabilities = torch.softmax(outputs, dim=1)  # Get probabilities
                _, predicted = torch.max(outputs, 1)

                loss = criterion(outputs, labels)
                val_loss += loss.item()
                all_probs.extend(probabilities.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_val_loss = val_loss / len(val_loader)
        avg_val_loss.append(epoch_val_loss)

        epoch_val_acc = correct / total
        avg_val_acc.append(epoch_val_acc)

        #print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {epoch_val_loss:.4f}, Val acc: {epoch_val_acc:.4f}")

        #if StepLR
        #scheduler.step()
        #reduce lr on plateau
        scheduler.step(val_loss)

        # Update the epoch progress bar
        epoch_pbar.update(1)
        

print("Finished training")

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from prettytable import PrettyTable

class Metrics:
    @staticmethod
    def calculate_accuracy(y_true, y_pred):
        return accuracy_score(y_true, y_pred)

    @staticmethod
    def calculate_precision(y_true, y_pred, average="weighted"):
        return precision_score(y_true, y_pred, average=average, zero_division=0)

    @staticmethod
    def calculate_recall(y_true, y_pred, average="weighted"):
        return recall_score(y_true, y_pred, average=average, zero_division=0)

    @staticmethod
    def calculate_f1_score(y_true, y_pred, average="weighted"):
        return f1_score(y_true, y_pred, average=average, zero_division=0)

    @staticmethod
    def calculate_confusion_matrix(y_true, y_pred, labels=None):
        return confusion_matrix(y_true, y_pred, labels=labels)

    @staticmethod
    def calculate_all(y_true, y_pred, average="macro"):
        table = PrettyTable(["Metric", "Value"])
        table.add_row(["Accuracy", Metrics.calculate_accuracy(y_true, y_pred)])
        table.add_row(["Precision", Metrics.calculate_precision(y_true, y_pred, average=average)])
        table.add_row(["Recall", Metrics.calculate_recall(y_true, y_pred, average=average)])
        table.add_row(["F1 Score", Metrics.calculate_f1_score(y_true, y_pred, average=average)])

        return table


In [ ]:
def train_val_metrics(avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, num_epochs):
    table = PrettyTable()

    table.field_names = ['Epoch' , "Train Loss", "Train Accuracy", "Validation Loss", "Validation Accuracy"]

    for epoch in range(num_epochs):
        table.add_row([epoch + 1, avg_train_loss[epoch], avg_train_acc[epoch], avg_val_loss[epoch], avg_val_acc[epoch]])

    print("\n Training & Validation Metrics:")
    print(table)

    return str(table)
    
table = train_val_metrics(avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, num_epochs)

print(Metrics.calculate_all(all_labels, all_preds))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
import numpy as np

all_labels_binarized = label_binarize(all_labels, classes=range(num_classes))
auc_scores = []
for i in range(num_classes):
    auc = roc_auc_score(all_labels_binarized[:, i], np.array(all_probs)[:, i])
    auc_scores.append(auc)

micro_auc = roc_auc_score(all_labels_binarized, np.array(all_probs), average="micro")
macro_auc = roc_auc_score(all_labels_binarized, np.array(all_probs), average="macro")

print(f"Per-class AUC Scores: {auc_scores}")
print(f"Micro-average AUC: {micro_auc}")
print(f"Macro-average AUC: {macro_auc}")



In [ ]:

import matplotlib.pyplot as plt
import datetime
def plot_roc(all_labels_binarized, all_probs, auc_scores):
    plt.figure(figsize=(12, 8))

    for i in range(len(auc_scores)):
        fpr, tpr, _ = roc_curve(all_labels_binarized[:, i], np.array(all_probs)[:, i])
        plt.plot(fpr, tpr, label=f"Class {i} (AUC = {auc_scores[i]:.2f})")

    # Add diagonal line
    plt.plot([0, 1], [0, 1], 'k--', lw=2)

    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    fig = plt.gcf()
    fig.tight_layout()
    plt.show()
    return fig  
roc_fig = plot_roc(all_labels_binarized,all_probs,auc_scores)



In [ ]:
def plot_train_val(num_epochs, avg_train_loss, avg_val_loss, avg_train_acc, avg_val_acc):
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(12, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, avg_train_loss, 'b-', label='Train Loss')
    plt.plot(epochs, avg_val_loss, 'r-', label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train and Validation Loss')
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, avg_train_acc, 'b-', label='Train Accuracy')
    plt.plot(epochs, avg_val_acc, 'r-', label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Train and Validation Accuracy')
    plt.legend()
    fig = plt.gcf()

    fig.tight_layout()
    plt.show()
    return fig
train_val_fig = plot_train_val(num_epochs, avg_train_loss, avg_val_loss, avg_train_acc, avg_val_acc)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

def plot_confusion_matrix(all_labels, all_preds, labels):
    labels = [label_to_index[label] for label in label_to_index.keys()]
    cm = Metrics.calculate_confusion_matrix(all_labels, all_preds, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(label_to_index.keys()))
    disp.plot(cmap='viridis')
    fig = plt.gcf()
    fig.tight_layout()
    plt.show()
    return fig   
cm_fig = plot_confusion_matrix(all_labels, all_preds, labels=filtered_data['Finding Labels'].unique())

In [ ]:
def savefigs(cm_fig, roc_fig, train_val_fig, run_name, results_file):
    # Define file paths
    cm_plot_file = f"DNNProject/runs/confusion_matrices/confusion_matrix_{run_name}.png"
    roc_file = f"DNNProject/runs/roc_curve_{run_name}.png"
    train_val_plot_file = f"DNNProject/runs/loss_accuracy_plots/loss_accuracy_plot_{run_name}.png"

    # Save Confusion Matrix
    try:
        cm_fig.savefig(cm_plot_file, bbox_inches='tight')
        print(f"Saved Confusion Matrix: {cm_plot_file}")
    except Exception as e:
        print(f"Failed to save Confusion Matrix: {e}")
    finally:
        plt.close(cm_fig)

    # Save ROC Curve
    try:
        roc_fig.savefig(roc_file, bbox_inches='tight')
        print(f"Saved ROC Curve: {roc_file}")
    except Exception as e:
        print(f"Failed to save ROC Curve: {e}")
    finally:
        plt.close(roc_fig)

    # Save Train-Validation Plot
    try:
        train_val_fig.savefig(train_val_plot_file, bbox_inches='tight')
        print(f"Saved Train-Val Plot: {train_val_plot_file}")
    except Exception as e:
        print(f"Failed to save Train-Val Plot: {e}")
    finally:
        plt.close(train_val_fig)

    # Log files in results file
    with open(results_file, 'a') as f:
        f.write(f"\n Confusion Matrix: {cm_plot_file}")
        f.write(f"\n ROC Curve: {roc_file}")
        f.write(f"\n Train-Val Plot: {train_val_plot_file}")

In [ ]:

import datetime
run_name = f"run_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S%f')}"  # Add microseconds

# Log file
results_file = 'DNNProject/runs/results_with_ROC.txt'

with open(results_file, 'a') as f:
    f.write("\n" + "="*50 + "\n")  # Separator mellom kjøringer
    f.write(f"Run: {run_name}\n")
    f.write(f"Model: {model.resnetModel}\n")
    
    f.write("Transforms:\n")
    for transform in train_transforms.transforms:
        f.write(f"  - {transform}\n")
    if 'scheduler' in globals() or 'scheduler' in locals():
        if scheduler is not None:
            scheduler_exist = True
            f.write("Scheduler:\n")
            f.write(f"  Type: {type(scheduler).__name__}\n")
            if hasattr(scheduler, 'get_last_lr'):
                f.write(f"  Initial LR: {scheduler.get_last_lr()}\n")
            if isinstance(scheduler, torch.optim.lr_scheduler.StepLR):
                f.write(f"  Step Size: {scheduler.step_size}\n")
                f.write(f"  Gamma: {scheduler.gamma}\n")
            elif isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                f.write(f"  Mode: {scheduler.mode}\n")
                f.write(f"  Patience: {scheduler.patience}\n")

    else:
        scheduler_exist = False
        f.write("Scheduler: None\n")
    f.write(f"Epochs: {num_epochs}\n")
    f.write(f"Optimizer:{optimizer.__class__.__name__} \n ")
    f.write("Hyperparameters:\n")
    if optimizer.__class__.__name__ == 'SGD':
        for i, param_group in enumerate(optimizer.param_groups):
            f.write(f"  Parameter Group {i + 1}:\n")
            f.write(f"    Learning Rate: {param_group['lr']} \n")
            f.write(f"    Weight Decay: {param_group['weight_decay']} \n")
            if 'momentum' in param_group:
                f.write(f"    Momentum: {param_group['momentum']} \n")
            else:
                f.write(f"    Momentum: Not Available\n")
    elif optimizer.__class__.__name__ == 'Adam':
        for i, param_group in enumerate(optimizer.param_groups):
            f.write(f"  Parameter Group {i + 1}:\n")
            f.write(f"    Learning Rate: {param_group['lr']} \n")
            if 'betas' in param_group:
                f.write(f"    Betas: {param_group['betas']}\n")
            else:
                f.write(f"    Betas: Not Available\n")
            f.write(f"    Weight Decay: {param_group['weight_decay']} \n")

    f.write("\n Training & Validation Metrics:\n")
    f.write(table + "\n")
    
    metrics = str(Metrics.calculate_all(all_labels, all_preds))
    f.write("\n Overall Metrics:\n")
    f.write(metrics + "\n")

    f.write("\n AUC-ROC Metrics:\n")
    f.write(f"Per-class AUC Scores: {auc_scores}\n")
    f.write(f"Micro-average AUC: {micro_auc:.4f}\n")
    f.write(f"Macro-average AUC: {macro_auc:.4f}\n")
    f.write(f"Plots:")
    savefigs(cm_fig,roc_fig,train_val_fig,run_name,results_file)